In [1]:
pip list | grep torch

torch                     2.2.2+cu121
torchvision               0.17.2+cu121
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

True
12.1
NVIDIA GeForce RTX 5070


/home/nathan/.pyenv/versions/hackathon/lib/python3.10/site-packages/torch/cuda/__init__.py:218: UserWarning: 
NVIDIA GeForce RTX 5070 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Simple 1D -> 1D CNN

In [3]:
import torch
from torch.utils.data import Dataset
from torch import FloatTensor

BATCH_SIZE = 1
INPUT_VEC_SIZE = 4 # how many columns per row
OUTPUT_VEC_SIZE_GV = 2001 # global view output vector size

# relu for hidden
# sigmoid for output

class TabularDataset(Dataset):

    def __init__(self, X:FloatTensor, Y:FloatTensor):
        """
        X: tensor with shape of (BATCH_SIZE, INPUT_VEC_SIZE)
        Y: tensor with shape of (BATCH_SIZE, OUTPUT_VEC_SIZE_GV)
        """

        self.X:FloatTensor = X 
        self.Y:FloatTensor = Y

    def __len__(self): return self.X.shape[0]

    def __getitem__(self, i):
        x = self.X[i].unsqueeze(0)
        if self.Y is None: return x
        return x, self.Y[i]


    

In [ ]:
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self, Y, hidden=64, kernel_size=1):
        super().__init__()
        pad = kernel_size // 2
        
        self.features = nn.Sequential(
            nn.Conv1d(1, hidden, kernel_size=kernel_size, padding=pad),
            nn.ReLU(),
            nn.Conv1d(hidden, hidden, kernel_size=kernel_size, padding=pad),
            nn.ReLU(),
        )

        self.resize = nn.AdaptiveAvgPool1d(Y)
        self.head = nn.Conv1d(hidden, 1, kernel_size=1)

    def forward(self, x):
        z = self.features(x)
        z = self.resize(z)
        z = self.head(z)
        out = z.squeeze(1)
        return out

In [10]:
from torch.utils.data import DataLoader

# assemble fake data for testing

fake_input = torch.randn(BATCH_SIZE, INPUT_VEC_SIZE)
fake_targets = torch.randn(BATCH_SIZE, OUTPUT_VEC_SIZE_GV)

loader = DataLoader(TabularDataset(fake_input, fake_targets), batch_size=2, shuffle=False)


In [11]:
model = CNN1D(Y=OUTPUT_VEC_SIZE_GV, hidden=16, kernel_size=1)
criterion = nn.MSELoss()
opt = torch.optim.Adam(model.parameters(), lr =1e-3)

In [12]:
import sys, torch, inspect
print("torch version:", torch.__version__)
print("torch file   :", torch.__file__)
import torch._utils_internal as ui
print("has justknobs_check?:", hasattr(ui, "justknobs_check"))

torch version: 2.2.2+cu121
torch file   : /home/nathan/.pyenv/versions/hackathon/lib/python3.10/site-packages/torch/__init__.py
has justknobs_check?: False


In [13]:
ls -la | grep -E '^d| torch.py'

drwxrwxr-x 3 nathan nathan    4096 Oct  4 13:22 ./
drwxrwxr-x 5 nathan nathan    4096 Oct  4 12:27 ../
drwxrwxr-x 8 nathan nathan    4096 Oct  4 13:22 .git/


In [14]:
for xb, yb in loader:
    pred = model(xb)                 # prints shapes
    loss = criterion(pred, yb)
    print("loss   :", loss.item())

    opt.zero_grad()
    loss.backward()                  # autograd computes grads
    opt.step()
    print("--- batch done ---\n")

AttributeError: 'CNN1D' object has no attribute 'feat'